In [1]:
import pandas as pd
import pickle
filename = 'finalized_model.sav'
clf = pickle.load(open(filename, 'rb'))

In [2]:
clf

RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=5,
                       n_estimators=200, random_state=42)

In [3]:
data = pd.read_csv("today.csv")
data["B365H"] = data["HomeDecimal"].astype("float")
data["B365D"] = data["DrawDecimal"].astype("float")
data["B365A"] = data["AwayDecimal"].astype("float")
data

,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A
0,Emelec,Atletico Mineiro,425,250,-134,5.250000,3.5,1.746269,5.250000,3.5,1.746269
1,Athletico Paranaense,Libertad Asuncion,-120,230,375,1.833333,3.3,4.750000,1.833333,3.3,4.750000
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.8,3.200000,2.550000,2.8,3.200000
3,Nacional De Football,Union Santa Fe,130,210,240,2.300000,3.1,3.400000,2.300000,3.1,3.400000
4,Colo Colo,Internacional,110,240,250,2.100000,3.4,3.500000,2.100000,3.4,3.500000
...,...,...,...,...,...,...,...,...,...,...,...
216,Liverpool (Calvin) Esports,Bayern (YoungDaddy) Esports,220,240,-106,3.200000,3.4,1.943396,3.200000,3.4,1.943396
217,Man Utd (Yaloo) Esports,Real Madrid (Honey) Esports,-110,260,220,1.909091,3.6,3.200000,1.909091,3.6,3.200000
218,A.Madrid (myxlunka) Esports,Chelsea (Bluefir3) Esports,850,600,-600,9.500000,7.0,1.166667,9.500000,7.0,1.166667
219,Chelsea (Krouks) Esports,Tottenham (Nasmi) Esports,-125,350,200,1.800000,4.5,3.000000,1.800000,4.5,3.000000


In [4]:
import numpy as np
predictProba =[]
Proba = []
df = data[["B365H","B365D","B365A"]]
for el in clf.predict_proba(df):
  choose = max(el)
  indexChoice = list(el).index(choose)

  if indexChoice == 0:
    predictProba.append("A")
  if indexChoice == 1:
    predictProba.append("D")
  if indexChoice == 2:
    predictProba.append("H")
  
  Proba.append(choose)

In [5]:
data["bet"] = predictProba
data["Proba"] = Proba
data

,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A,bet,Proba
0,Emelec,Atletico Mineiro,425,250,-134,5.250000,3.5,1.746269,5.250000,3.5,1.746269,A,0.444928
1,Athletico Paranaense,Libertad Asuncion,-120,230,375,1.833333,3.3,4.750000,1.833333,3.3,4.750000,H,0.539746
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.8,3.200000,2.550000,2.8,3.200000,H,0.483421
3,Nacional De Football,Union Santa Fe,130,210,240,2.300000,3.1,3.400000,2.300000,3.1,3.400000,H,0.383085
4,Colo Colo,Internacional,110,240,250,2.100000,3.4,3.500000,2.100000,3.4,3.500000,H,0.472432
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Liverpool (Calvin) Esports,Bayern (YoungDaddy) Esports,220,240,-106,3.200000,3.4,1.943396,3.200000,3.4,1.943396,A,0.367434
217,Man Utd (Yaloo) Esports,Real Madrid (Honey) Esports,-110,260,220,1.909091,3.6,3.200000,1.909091,3.6,3.200000,H,0.510435
218,A.Madrid (myxlunka) Esports,Chelsea (Bluefir3) Esports,850,600,-600,9.500000,7.0,1.166667,9.500000,7.0,1.166667,A,0.804572
219,Chelsea (Krouks) Esports,Tottenham (Nasmi) Esports,-125,350,200,1.800000,4.5,3.000000,1.800000,4.5,3.000000,H,0.624345


In [6]:
def AddKellyCriterion(row):
    bet = row["bet"]
    if bet == "H":
        odd = row["B365H"]
    elif bet == "D": 
        odd = row["B365D"] 
    else: 
        odd = row["B365A"]
        
    B = odd - 1
    P = row["Proba"]
    Q = 1- P
    row["Kelly"] = (B*P - Q)/B
    return row

data = data.apply(AddKellyCriterion,axis =1)
data

,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A,bet,Proba,Kelly
0,Emelec,Atletico Mineiro,425,250,-134,5.250000,3.5,1.746269,5.250000,3.5,1.746269,A,0.444928,-0.298868
1,Athletico Paranaense,Libertad Asuncion,-120,230,375,1.833333,3.3,4.750000,1.833333,3.3,4.750000,H,0.539746,-0.012559
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.8,3.200000,2.550000,2.8,3.200000,H,0.483421,0.150145
3,Nacional De Football,Union Santa Fe,130,210,240,2.300000,3.1,3.400000,2.300000,3.1,3.400000,H,0.383085,-0.091465
4,Colo Colo,Internacional,110,240,250,2.100000,3.4,3.500000,2.100000,3.4,3.500000,H,0.472432,-0.007176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Liverpool (Calvin) Esports,Bayern (YoungDaddy) Esports,220,240,-106,3.200000,3.4,1.943396,3.200000,3.4,1.943396,A,0.367434,-0.303086
217,Man Utd (Yaloo) Esports,Real Madrid (Honey) Esports,-110,260,220,1.909091,3.6,3.200000,1.909091,3.6,3.200000,H,0.510435,-0.028086
218,A.Madrid (myxlunka) Esports,Chelsea (Bluefir3) Esports,850,600,-600,9.500000,7.0,1.166667,9.500000,7.0,1.166667,A,0.804572,-0.367995
219,Chelsea (Krouks) Esports,Tottenham (Nasmi) Esports,-125,350,200,1.800000,4.5,3.000000,1.800000,4.5,3.000000,H,0.624345,0.154776


In [7]:
print(len(data[data["Kelly"] > 0])/len(data))
data = data[data["Kelly"] > 0]
data

0.38009049773755654


,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A,bet,Proba,Kelly
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.80,3.20,2.550000,2.80,3.20,H,0.483421,0.150145
6,Gimnasia LP,CSD Flandria,-200,300,650,1.500000,4.00,7.50,1.500000,4.00,7.50,H,0.707862,0.123586
12,Grange Thistle,Souths United,137,260,140,2.370000,3.60,2.40,2.370000,3.60,2.40,A,0.457835,0.070574
13,Adelaide Olympic,Adelaide United NPL,-138,300,275,1.724638,4.00,3.75,1.724638,4.00,3.75,H,0.703124,0.293436
16,Brusque,Bahia,200,175,187,3.000000,2.75,2.87,3.000000,2.75,2.87,H,0.487562,0.231343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Inter (DaVa) Esports,Barcelona (Bomb1to) Esports,260,105,200,3.600000,2.05,3.00,3.600000,2.05,3.00,H,0.413188,0.187491
211,PSG (Nederfox) Esports,Liverpool (Calvin) Esports,-138,275,275,1.724638,3.75,3.75,1.724638,3.75,3.75,H,0.662108,0.195817
212,A.Madrid (Kodak) Esports,Inter (DaVa) Esports,180,200,137,2.800000,3.00,2.37,2.800000,3.00,2.37,D,0.463140,0.194710
215,Napoli (dm1trena) Esports,Lyon (DriLLeR) Esports,120,225,187,2.200000,3.25,2.87,2.200000,3.25,2.87,H,0.486940,0.059389


In [8]:
import numpy as np
def softmax(vector):
    e = np.exp(vector)
    return e / e.sum()
 
data["betDatePortion"]= softmax(data["Kelly"])
data

c:\users\khoas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A,bet,Proba,Kelly,betDatePortion
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.80,3.20,2.550000,2.80,3.20,H,0.483421,0.150145,0.011787
6,Gimnasia LP,CSD Flandria,-200,300,650,1.500000,4.00,7.50,1.500000,4.00,7.50,H,0.707862,0.123586,0.011478
12,Grange Thistle,Souths United,137,260,140,2.370000,3.60,2.40,2.370000,3.60,2.40,A,0.457835,0.070574,0.010886
13,Adelaide Olympic,Adelaide United NPL,-138,300,275,1.724638,4.00,3.75,1.724638,4.00,3.75,H,0.703124,0.293436,0.013603
16,Brusque,Bahia,200,175,187,3.000000,2.75,2.87,3.000000,2.75,2.87,H,0.487562,0.231343,0.012784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Inter (DaVa) Esports,Barcelona (Bomb1to) Esports,260,105,200,3.600000,2.05,3.00,3.600000,2.05,3.00,H,0.413188,0.187491,0.012236
211,PSG (Nederfox) Esports,Liverpool (Calvin) Esports,-138,275,275,1.724638,3.75,3.75,1.724638,3.75,3.75,H,0.662108,0.195817,0.012338
212,A.Madrid (Kodak) Esports,Inter (DaVa) Esports,180,200,137,2.800000,3.00,2.37,2.800000,3.00,2.37,D,0.463140,0.194710,0.012325
215,Napoli (dm1trena) Esports,Lyon (DriLLeR) Esports,120,225,187,2.200000,3.25,2.87,2.200000,3.25,2.87,H,0.486940,0.059389,0.010765


In [9]:
from datetime import datetime
data["Date"] = datetime.today().strftime('%Y-%m-%d')
data

c:\users\khoas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A,bet,Proba,Kelly,betDatePortion,Date
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.80,3.20,2.550000,2.80,3.20,H,0.483421,0.150145,0.011787,2022-06-28
6,Gimnasia LP,CSD Flandria,-200,300,650,1.500000,4.00,7.50,1.500000,4.00,7.50,H,0.707862,0.123586,0.011478,2022-06-28
12,Grange Thistle,Souths United,137,260,140,2.370000,3.60,2.40,2.370000,3.60,2.40,A,0.457835,0.070574,0.010886,2022-06-28
13,Adelaide Olympic,Adelaide United NPL,-138,300,275,1.724638,4.00,3.75,1.724638,4.00,3.75,H,0.703124,0.293436,0.013603,2022-06-28
16,Brusque,Bahia,200,175,187,3.000000,2.75,2.87,3.000000,2.75,2.87,H,0.487562,0.231343,0.012784,2022-06-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Inter (DaVa) Esports,Barcelona (Bomb1to) Esports,260,105,200,3.600000,2.05,3.00,3.600000,2.05,3.00,H,0.413188,0.187491,0.012236,2022-06-28
211,PSG (Nederfox) Esports,Liverpool (Calvin) Esports,-138,275,275,1.724638,3.75,3.75,1.724638,3.75,3.75,H,0.662108,0.195817,0.012338,2022-06-28
212,A.Madrid (Kodak) Esports,Inter (DaVa) Esports,180,200,137,2.800000,3.00,2.37,2.800000,3.00,2.37,D,0.463140,0.194710,0.012325,2022-06-28
215,Napoli (dm1trena) Esports,Lyon (DriLLeR) Esports,120,225,187,2.200000,3.25,2.87,2.200000,3.25,2.87,H,0.486940,0.059389,0.010765,2022-06-28


In [10]:
data["runningMoney"] = 10
data["BetAmount"] = data["betDatePortion"] * data["runningMoney"]
data["refill"] = 0
data["withdraw"] = 0
data

c:\users\khoas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\khoas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\khoas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,HomeTeam,AwayTeam,Home,Draw,Away,HomeDecimal,DrawDecimal,AwayDecimal,B365H,B365D,B365A,bet,Proba,Kelly,betDatePortion,Date,runningMoney,BetAmount,refill,withdraw
2,Corinthians,Boca Juniors,155,180,220,2.550000,2.80,3.20,2.550000,2.80,3.20,H,0.483421,0.150145,0.011787,2022-06-28,10,0.117874,0,0
6,Gimnasia LP,CSD Flandria,-200,300,650,1.500000,4.00,7.50,1.500000,4.00,7.50,H,0.707862,0.123586,0.011478,2022-06-28,10,0.114785,0,0
12,Grange Thistle,Souths United,137,260,140,2.370000,3.60,2.40,2.370000,3.60,2.40,A,0.457835,0.070574,0.010886,2022-06-28,10,0.108858,0,0
13,Adelaide Olympic,Adelaide United NPL,-138,300,275,1.724638,4.00,3.75,1.724638,4.00,3.75,H,0.703124,0.293436,0.013603,2022-06-28,10,0.136035,0,0
16,Brusque,Bahia,200,175,187,3.000000,2.75,2.87,3.000000,2.75,2.87,H,0.487562,0.231343,0.012784,2022-06-28,10,0.127845,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Inter (DaVa) Esports,Barcelona (Bomb1to) Esports,260,105,200,3.600000,2.05,3.00,3.600000,2.05,3.00,H,0.413188,0.187491,0.012236,2022-06-28,10,0.122360,0,0
211,PSG (Nederfox) Esports,Liverpool (Calvin) Esports,-138,275,275,1.724638,3.75,3.75,1.724638,3.75,3.75,H,0.662108,0.195817,0.012338,2022-06-28,10,0.123383,0,0
212,A.Madrid (Kodak) Esports,Inter (DaVa) Esports,180,200,137,2.800000,3.00,2.37,2.800000,3.00,2.37,D,0.463140,0.194710,0.012325,2022-06-28,10,0.123246,0,0
215,Napoli (dm1trena) Esports,Lyon (DriLLeR) Esports,120,225,187,2.200000,3.25,2.87,2.200000,3.25,2.87,H,0.486940,0.059389,0.010765,2022-06-28,10,0.107648,0,0


In [11]:
data.to_csv("masterSheet.csv",index = False)

In [115]:
# -------------------------------------------------------------------------

In [116]:
# list of names
runningMoney = [10]

with open('runningMoney.txt', 'w') as f:
    for item in runningMoney:
        f.write("%s\n" % item)

In [117]:
with open('runningMoney.txt') as f:
    lines = f.readlines()
lines

['10\n']